In [ ]:
import os
import requests
import dotenv
import base64 # used to encode strings into bytes and back
import json # used to convert JSON data into strings
import webbrowser
from urllib.parse import urlencode

# Credits to original article https://python.plainenglish.io/bored-of-libraries-heres-how-to-connect-to-the-spotify-api-using-pure-python-bd31e9e3d88a

In [2]:
dotenv.load_dotenv()
client_id = os.getenv("client_id")
client_secret = os.getenv("client_secret")
# print(client_id, client_secret)

True

In [55]:
## Step 1
# This is necessary to get authorization code, we run the function below to open web browser, and extract the code in URL after authentication
# This auth code will then be used in step 2

def get_auth_code():
    auth_headers = {
    "client_id": os.getenv("client_id"),
    "response_type": "code",
    "redirect_uri": "https://localhost:3000", # Based on Spotify web app setting redirect url
    "scope": 'user-read-recently-played' # Specify the auth code scope. Refer https://developer.spotify.com/documentation/web-api/concepts/scopes#user-read-recently-played
    }
    webbrowser.open("https://accounts.spotify.com/authorize?" + urlencode(auth_headers))

get_auth_code()



In [56]:
code = 'AQCXXY1_eIc8ILHwEobU1XxMSe8wRI2poQ3y6LJ3MPDqbKVJrWCGoYd0-yVFwQIfkmKbGpNsszb8bK1ks5V8oFvu79xItBX9hni1DhEaPvqN9ks175QvBGrXlO4skKLGcFgNm6MVZpgahx2BjseT_Ssbh_FB2Q0-3UeGMnvGg-n6xoSLxgWbE2M62Zco4xAWDyezO2E'

In [57]:
## Step 2
# We pass the auth code above into the function below to get Bearer token 
def get_bearer_token():
    encoded_credentials = base64.b64encode(os.getenv('client_id').encode() + b':' + os.getenv('client_secret').encode()).decode("utf-8")

    token_headers = {
        "Authorization": "Basic " + encoded_credentials,
        "Content-Type": "application/x-www-form-urlencoded"
    }

    token_data = {
        "grant_type": "authorization_code",
        "code": code,
        "redirect_uri": "https://localhost:3000"
    }

    r = requests.post("https://accounts.spotify.com/api/token", data=token_data, headers=token_headers)
    return r.content

In [58]:
# Only run this once per auth code, if we rerun again it will return error
token_details = json.loads(get_bearer_token())
token_details['access_token']

In [61]:
## Step 3a
# This is the function to retrieve our data in Spotify
# Scope = "user-library-read"
def get_user_libs():
    url = "https://api.spotify.com/v1/me/tracks"
    # headers = get_auth_header(token_details['access_token']) # Authorization: Bearer ....

    user_headers = {
    "Authorization": "Bearer " + token_details['access_token'],
    "Content-Type": "application/json"
}
    
    user_params = {"limit": 50}

    return requests.get(url, params=user_params, headers=user_headers)

json.loads(get_user_libs().content)

{'error': {'status': 403, 'message': 'Forbidden.'}}

In [68]:
## Step 3b
# This is the function to retrieve our data in Spotify
# Scope = "user-read-recently-played"
def get_recently_played():    
    headerr = {"Authorization" : "Bearer " + token_details['access_token']}  

    return requests.get('https://api.spotify.com/v1/me/player/recently-played', 
        headers=headerr
        ).content

tracks = json.loads(get_recently_played())['items']

In [78]:
print("ARTIST NAME:" ,tracks[0]['track']['artists'][0]['name'])
print("TRACK NAME:", tracks[0]['track']['name'])

ARTIST NAME: Satin Jackets
TRACK NAME: Out of My Head
